# Managing conversations with a GPT chat model
This sample notebook demonstrates a couple of simple patterns you can use for managing a continuous conversation and its message history with a GPT chat model. It builds on the basic chat completion example shown in the previous notebook.

## 1. Setting up the environment

We again start by installing and importing the required libraries and setting up the API configuration. In addition to the `openai` library from the previous notebook, we also import the `tiktoken` library, which we will use to manage the length of the message history we keep. This is necessary to stay within the model's token limit.

In [8]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai

In [9]:
# if needed, install and/or upgrade to the latest version of the tiktoken Python library
%pip install --upgrade tiktoken

In [10]:
# import the OpenAI and tiktoken Python library
import openai
import tiktoken

In [11]:
# setup parameters for using an Azure OpenAI endpoint
openai.api_type = "azure"
openai.api_key = "<your_api_key>"                           # The API key for your Azure OpenAI resource -> Get yours from https://ai.cosmoconsult.com/me
openai.api_base = "https://apis.ai.cosmoconsult.com"        # The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_version = "2023-07-01-preview"                   # The API version for your Azure OpenAI resource. e.g. "2023-07-01-preview"

## 2. Defining helper functions

To successfully manage a conversation with a GPT chat model, we want the model to be aware of the message history. However, since each model has a limit on the number of tokens it can process (and thereby the length of the message history), we need to make sure that we don't exceed this limit. To be able to do this, we use the `tiktoken` library to keep track of the number of tokens in the message history and remove the oldest messages when the limit is reached.
The number of tokens that can be processed depends on the model used. For example, `gpt-3.5-turbo` can process up to 4097 tokens, while `gpt-4` is capable of processing up to 8192 tokens. A good rule of thumb is to keep the message history below 80% of the model's token limit, to allow for some flexibility in the length of the generated response. A complete overview of the token limits for each model can be found in the [OpenAI Models Overview](https://platform.openai.com/docs/models/gpt-4).

In [12]:
def num_tokens_from_messages(messages):
    encoding = tiktoken.get_encoding("cl100k_base") # load the encoding to use for counting the tokens. The "cl100k_base" encoding is compatibles with the OpenAI models gpt-4, gpt-3.5-turbo and text-embedding-ada-002
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

## 3. Creating a continuous conversation

To run a conversation, we are now adding the models response and the user input to the message list in a continuous loop. To avoid running out of tokens, we are checking the number of tokens in the message list each time before requesting the next response from the model. If the number of tokens exceeds our defined limit, we keep removing the oldest messages from the list until we are below the limit again. <br>
This is of course a very basic example of how to manage a continuous conversation with a GPT chat model. Depending on your use case, you might want to add additional logic to the conversation loop, for example implementing a memory management in which older messages are not simply removed, but summarized or merged with newer messages, or even more sophisticated methods that extract and store contents from the message history in separate memories where they can be retrieved from on demand.

In [13]:
# Define the maximum number of tokens allowed in the conversation
max_tokens = 3000

# Define the system message
system_message = "You are a helpful assistant."

# Initialize the message history with the system message
messages = [{"role": "system", "content": system_message}]

In [14]:
from time import sleep

# Read the first message from the user
user_message = input("You: ")

# Create a conversation loop
while user_message != "exit":   # The loop will run until the user types "exit"
    
    # Add the user message to the message history
    messages.append({"role": "user", "content": user_message})
    print("[USER]: " + user_message)

    # Manage the length of the message history
    while num_tokens_from_messages(messages) > max_tokens:
        messages.pop(1) # remove the oldest message (but keep the system message at index 0)
    
    # Generate a response from the GPT model
    response = openai.ChatCompletion.create(
        deployment_id = "gpt-35-turbo", # The deployment ID for your Azure OpenAI resource
        messages = messages,
        temperature = 0.1,
        max_tokens = 150
    )

    # Add the response to the message history
    assistant_message = response['choices'][0]['message']['content']
    messages.append({"role": "assistant", "content": assistant_message})
    print("[ASSISTANT]: " + assistant_message)

    # Read the next message from the user
    sleep(1)
    user_message = input("You: ")


[USER]: Who was Marie Curie?
[ASSISTANT]: Marie Curie, born Maria Skłodowska, was a Polish-born physicist and chemist who became one of the most famous scientists in history. She is best known for her pioneering research on radioactivity, which led to the discovery of two new elements, polonium and radium. Curie was the first woman to win a Nobel Prize and remains the only person to have won Nobel Prizes in two different scientific fields (Physics and Chemistry). Her groundbreaking work laid the foundation for modern physics and greatly contributed to advancements in medical science. Curie's achievements and her determination to overcome gender barriers have made her an inspirational figure for generations of scientists.
[USER]: Did she met Albert Einstein?
[ASSISTANT]: Yes, Marie Curie did meet Albert Einstein. In 1911, Curie attended the first Solvay Conference in Brussels, which was a gathering of the world's leading physicists. Einstein was also present at the conference, and it is

Again, feel free to experiment with the code! Try out how the model responds to different inputs and system messages and see how the conversation changes when you change the model or the parameters. Are there ways to keep track of information set gets removed due to the message list token limit? Or can you make two models with different personalities talk to each other?

Have fun!